In [1]:
import json
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
from time import time
import io
import re

import pickle
from csv import reader
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from tensorflow.keras import layers
from tensorflow.keras.backend import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import plot_model

# physical_devices = tf.config.list_physical_devices('GPU') 
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

from datetime import datetime
from collections import defaultdict

from sklearn.model_selection import train_test_split



# Data

In [2]:
train_section = pd.read_table("Article-Bias-Prediction-main/data/randomtraindata.csv", sep=',', index_col=0)
test_section = pd.read_table("Article-Bias-Prediction-main/data/randomtestdata.csv", sep=',', index_col=0)
val_section = pd.read_table("Article-Bias-Prediction-main/data/randomtestdata.csv", sep=',', index_col=0)


In [3]:
test_section = pd.read_table("test_dataset.csv",  index_col=0)

test_section.head()

,index,url,q3,perceived,primary.topic,secondary.topic,democrat.vote,republican.vote,article,length,democrat.bias,republican.bias,biaslabel,isbiased,biascategories,bias_magnitude,bias_text,content,pred_labels
0,2,http://www.washingtonpost.com/opinions/dont-le...,Opinion,1,Civil Rights,Civil Rights,Neutral,Neutral,david cole is a professor of constitutional la...,600,0.0,0.0,Neutral,0,0,0.0,center,david cole is a professor of constitutional la...,1
1,4,http://www.breitbart.com/Big-Government/2013/1...,Opinion,1,Civil Rights,Civil Rights,Neutral,Neutral,u.s. immigration citizenship and immigration s...,600,0.0,0.0,Neutral,0,0,0.0,center,u.s. immigration citizenship and immigration s...,2
2,8,http://www.dailykos.com/story/2013/12/02/12595...,News,1,Civil Rights,Civil Rights,SomewhatPositive,SomewhatNegative,the following irresponsible police...,498,0.5,-0.5,Liberal,1,1,1.0,left,the following irresponsible police...,2
3,9,http://www.nytimes.com/2013/05/24/us/politics/...,News,1,Civil Rights,Civil Rights,Neutral,Neutral,an alternative possibility is that a bipartisa...,372,0.0,0.0,Neutral,0,0,0.0,center,an alternative possibility is that a bipartisa...,1
4,11,http://www.latimes.com/news/local/la-me-cap-gu...,other,0,Civil Rights,Civil Rights,Neutral,SomewhatPositive,"sacramento — “living in parallel universes,” i...",600,0.0,0.5,Somewhat Conservative,1,-1,0.5,right,"sacramento — “living in parallel universes,” i...",1


In [4]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_labels = np.asarray(le.fit_transform(train_section['bias_text']))
val_labels = np.asarray(le.transform(val_section['bias_text']))
test_labels = np.asarray(le.transform(test_section['bias_text']))

In [5]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

## TF-IDF Label Prediction

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
# tf-idf processing from https://github.com/williamscott701/Information-Retrieval/blob/master/2.%20TF-IDF%20Ranking%20-%20Cosine%20Similarity%2C%20Matching%20Score/TF-IDF.ipynb

def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text


def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")


def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text


def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text


def preprocess(data):
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

In [12]:
# train_dataset = list(train_section["content"])
# val_dataset = list(val_section["content"])
test_dataset = list(test_section["content"])


In [11]:
def process(dataset):
    processed_text = []
    for i in dataset:
#         processed_text.append(word_tokenize(str(preprocess(i))))
        processed_text.append(str(preprocess(i)))
    return processed_text


In [8]:
import pickle

In [13]:
# train_dataset = process(train_dataset)
# val_dataset = process(val_dataset)
test_dataset = process(test_dataset)

# train_dataset = pickle.load(open("processed_train.pickle", "rb"))
# val_dataset = pickle.load(open("processed_val.pickle", "rb"))
# # test_dataset = pickle.load(open("processed_sourcetest.pickle", "rb"))

In [14]:
# # tf-idf from https://stackabuse.com/text-classification-with-python-and-scikit-learn/
# from sklearn.feature_extraction.text import CountVectorizer

# vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
# trainX = vectorizer.fit_transform(train_dataset).toarray()
# valX = vectorizer.transform(val_dataset).toarray()
# testX = vectorizer.transform(test_dataset).toarray()

# tf-idf from https://stackabuse.com/text-classification-with-python-and-scikit-learn/
from sklearn.feature_extraction.text import CountVectorizer

# vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
# trainX = vectorizer.fit_transform(train_dataset).toarray()
# valX = vectorizer.transform(val_dataset).toarray()
vectorizer = pickle.load(open("vectorizersourcelevel.pickle", "rb"))

testX = vectorizer.transform(test_dataset).toarray()

In [15]:
# from sklearn.feature_extraction.text import TfidfTransformer
# tfidfconverter = TfidfTransformer()
# trainX = tfidfconverter.fit_transform(trainX).toarray()
# valX = tfidfconverter.transform(valX).toarray()
# testX = tfidfconverter.transform(testX).toarray()

from sklearn.feature_extraction.text import TfidfTransformer
# tfidfconverter = TfidfTransformer()
# trainX = tfidfconverter.fit_transform(trainX).toarray()
# valX = tfidfconverter.transform(valX).toarray()
tfidfconverter = pickle.load(open("tfidfconverter.pickle", "rb"))

testX = tfidfconverter.transform(testX).toarray()

In [ ]:

# train_labels = np.asarray(le.transform(train_section['label']))
# val_labels = np.asarray(le.transform(val_section['label']))
# test_labels = np.asarray(le.transform(test_section['label']))

In [17]:
testX.shape

(8350, 1500)

In [18]:
# multibinary
def convertlabels(labelarray, target):
    labellist = list(labelarray)
    for i in range(len(labellist)):
        if labellist[i] !=target:
            labellist[i] = 0
        else:
            labellist[i] = 1
    return np.array(labellist)

In [19]:
# center vs else
centertrainlabels = convertlabels(trainval_labels, 0)
centertestlabels = convertlabels(test_labels, 0)

# left vs else
lefttrainlabels = convertlabels(trainval_labels, 1)
lefttestlabels = convertlabels(test_labels, 1)

# right vs else
righttrainlabels = convertlabels(trainval_labels, 2)
righttestlabels = convertlabels(test_labels, 2)


NameError: name 'trainval_labels' is not defined

In [20]:
print("0 = center, 1 = left, 2 = right")
print(np.bincount(trainval_labels))
plt.bar([0,1,2], np.bincount(trainval_labels))

0 = center, 1 = left, 2 = right


NameError: name 'trainval_labels' is not defined

In [ ]:
print("0 = other, 1 = center")
print(np.bincount(centertrainlabels))
plt.bar([0, 1], np.bincount(centertrainlabels))

In [ ]:
print("0 = other, 1 = left")
print(np.bincount(lefttrainlabels))
plt.bar([0, 1], np.bincount(lefttrainlabels))

In [ ]:
print("0 = other, 1 = right")
print(np.bincount(righttrainlabels))
plt.bar([0, 1], np.bincount(righttrainlabels))

In [21]:
centerclassifier = pickle.load(open("centerclassifier.pickle", "rb"))
leftclassifier = pickle.load(open("leftclassifier.pickle", "rb"))
rightclassifier = pickle.load(open("rightclassifier.pickle", "rb"))

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV



In [ ]:
# centerclassifier = RandomForestClassifier(n_estimators=1000, random_state=0)
# centerclassifier.fit(trainvalX, centertrainlabels) 

In [ ]:
# leftclassifier = RandomForestClassifier(n_estimators=1000, random_state=0)
# leftclassifier.fit(trainvalX, lefttrainlabels) 

In [ ]:
# rightclassifier = RandomForestClassifier(n_estimators=1000, random_state=0)
# rightclassifier.fit(trainvalX, righttrainlabels) 

In [ ]:
# # https://stackoverflow.com/questions/32764991/how-do-i-store-a-tfidfvectorizer-for-future-use-in-scikit-learn
# pickle.dump(centerclassifier, open("centerclassifier.pickle", "wb"))
# pickle.dump(leftclassifier, open("leftclassifier.pickle", "wb"))
# pickle.dump(rightclassifier, open("rightclassifier.pickle", "wb"))

In [23]:
classifier_list = [centerclassifier, leftclassifier, rightclassifier]
target_list = [0,1,2]

In [24]:
def multiclassifier(testdata, classifierlist, targetlist, reallabels):
    classified_probs = []
    for i in range(len(classifierlist)):
        probabilities = classifierlist[i].predict_proba(testdata)
        classified_probs.append(probabilities)
    print(np.array(classified_probs).shape)
    merged_classification = []
    for i in range(len(classified_probs[0])):
        itemproblist = []
        for ii in range(len(classifierlist)):
            itemproblist.append(classified_probs[ii][i][1])
        max_value = max(itemproblist)
        max_index = itemproblist.index(max_value)
        if reallabels[i] != itemproblist.index(max_value):
            print("you dun goofed")
        merged_classification.append(targetlist[max_index])
    print("done")
    return merged_classification

In [25]:
pred_labels = multiclassifier(testX, classifier_list, target_list, test_labels)

(3, 8350, 2)
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goofed
you dun goof

In [26]:
np.array(pred_labels).shape

(8350,)

In [27]:
test_labels.shape

(8350,)

In [28]:
pred_labels == test_labels

array([False, False, False, ..., False, False, False])

In [33]:
c = 0
for i in range(len(pred_labels)):
    if pred_labels[i] != test_labels[i]:
        c+=1
print(c)

5562


In [29]:
pickle.dump(pred_labels, open("mergedclassifier_pred_labels_article_level.pickle", "wb"))
pickle.dump(test_labels, open("mergedclassifier_test_labels_article_level.pickle", "wb"))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# param_grid = {
#                  'n_estimators': [100, 500, 1000],
#                  'max_depth': [2, 5, 9, 13]
#              }

param_grid = {
                 'n_estimators': [1000, 3000],
                 'max_depth': [2, 5, 13]
             }

rfc = RandomForestClassifier()

clf = GridSearchCV(rfc, param_grid, cv = 10)
clf.fit(trainX, train_labels)
sorted(clf.cv_results_.keys())
# classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
# classifier.fit(trainX, train_labels) 

In [ ]:
clf.best_params_

In [ ]:
import pickle

In [ ]:
# https://stackoverflow.com/questions/32764991/how-do-i-store-a-tfidfvectorizer-for-future-use-in-scikit-learn
pickle.dump(train_dataset, open("vectorized_train.pickle", "wb"))
pickle.dump(val_dataset, open("vectorized_val.pickle", "wb"))
pickle.dump(test_dataset, open("vectorized_sourcetest.pickle", "wb"))

In [ ]:
pickle.dump(train_dataset, open("vectorized_train.pickle", "wb"))
pickle.dump(val_dataset, open("vectorized_val.pickle", "wb"))
pickle.dump(test_dataset, open("vectorized_sourcetest.pickle", "wb"))

In [ ]:
pickle.dump(clf, open("clfsourcelevel.pickle", "wb"))

In [ ]:
pickle.dump(vectorizer, open("vectorizersourcelevel.pickle", "wb"))
pickle.dump(vectorizer, open("vectorizer.pickle", "wb"))
pickle.dump(tfidfconverter, open("tfidfconverter.pickle", "wb"))

In [ ]:
test_labels

#### TF-IDF Label Prediction

In [ ]:
y_pred = clf.predict(testX)
y_pred

In [ ]:
test_labels

In [ ]:
pickle.dump(y_pred, open("pred_y_labels_TFIDF_source.pickle", "wb"))
pickle.dump(test_labels, open("real_y_labels_TFIDF_source.pickle", "wb"))


In [ ]:
test_classes = y_pred
modelname = "TF-IDF"

print(modelname +" Metrics:")
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_labels, test_classes)
print('Accuracy: %f' % accuracy)
# modelaccuracylist.append(accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_labels, test_classes, "micro")
print('Precision (micro): %f' % precision)
# modelprecisionlist.append(precision)
# recall: tp / (tp + fn)
recall = recall_score(test_labels, test_classes)
print('Recall: %f' % recall)
# modelrecalllist.append(recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(test_labels, test_classes)
# modelf1list.append(f1)
print('F1 score: %f' % f1)
# mlist.append([accuracy, precision, recall, f1])
#     print(classification_report(test_labels,test_classes))
print("")

## Data Analysis

In [ ]:
t5_xlnet_predicted_labels = [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1]

In [ ]:
pegasus_xlnet_predicted_labels = [0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]

In [ ]:
text_xlnet_predicted_labels = test_section["XLNet"]

In [ ]:
t5_tfidf_predicted_labels = [0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1]

In [ ]:
pegasus_tfidf_predicted_labels = [1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
text_tfidf_predicted_labels = test_section["TFIDF"]

In [ ]:
test_labels = [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0]

In [ ]:
predlist = [t5_xlnet_predicted_labels, 
            pegasus_xlnet_predicted_labels, 
            text_xlnet_predicted_labels, 
            t5_tfidf_predicted_labels, 
            pegasus_tfidf_predicted_labels,
            text_tfidf_predicted_labels]

In [ ]:
namelist = ["XLNet (T5)",
           "XLNet (Pegasus)",
           "XLNet (Full Article)",
           "TF-IDF (T5)",
           "TF-IDF (Pegasus)",
           "TF-IDF (Full Article)"]

In [ ]:
for i in range(len(predlist)):
    predlist[i] = 1-np.array(predlist[i])
test_labels =1-np.array(test_labels)

In [ ]:
t5_xlnet_predicted_labels = predlist[0]
pegasus_xlnet_predicted_labels = predlist[1]
text_xlnet_predicted_labels = predlist[2]
t5_tfidf_predicted_labels = predlist[3]
pegasus_tfidf_predicted_labels = predlist[4]
text_tfidf_predicted_labels = predlist[5]

In [ ]:
test_section["pegasus_summaries"] = pegasus_summaries
test_section["t5_summaries"] = t5_summaries

In [ ]:
test_section['t5_xlnet_predicted_labels'] = t5_xlnet_predicted_labels
test_section['pegasus_xlnet_predicted_labels'] = pegasus_xlnet_predicted_labels
test_section['t5_tfidf_predicted_labels'] = t5_tfidf_predicted_labels
test_section['pegasus_tfidf_predicted_labels'] = pegasus_tfidf_predicted_labels
test_section['test_labels'] = test_labels
test_section.to_csv("test_dataset_final.csv", sep='\t')

In [ ]:
data["pegasus_summaries"] = pegasus_summaries

In [ ]:
data = pd.read_table("test_dataset_final.csv", sep='\t', index_col=0)
data.head()

In [ ]:
modelaccuracylist = []
modelnamelist = []
modelprecisionlist = []
modelrecalllist = []
modelf1list = []
mlist = []
print("Metrics for Biased as baseline class (0):")
print()
for i in range(len(predlist)):
    test_classes = predlist[i]
    modelname = namelist[i]
    modelnamelist.append(modelname)
    print(modelname +" Metrics:")
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(test_labels, test_classes)
    print('Accuracy: %f' % accuracy)
    modelaccuracylist.append(accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(test_labels, test_classes)
    print('Precision: %f' % precision)
    modelprecisionlist.append(precision)
    # recall: tp / (tp + fn)
    recall = recall_score(test_labels, test_classes)
    print('Recall: %f' % recall)
    modelrecalllist.append(recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(test_labels, test_classes)
    modelf1list.append(f1)
    print('F1 score: %f' % f1)
    mlist.append([accuracy, precision, recall, f1])
#     print(classification_report(test_labels,test_classes))
    print("")
 

In [ ]:
df = pd.DataFrame(columns = modelnamelist)
df.loc['Accuracy'] = modelaccuracylist
df.loc['Precision'] = modelprecisionlist
df.loc['Recall'] = modelrecalllist
df.loc['F1'] = modelf1list
df.head()

In [ ]:
ax = df.plot.bar(rot=0)
ax.set_ylim(ymin=0.3)
ax.figure.set_size_inches(6.5, 7)
plt.title('Metric Comparison')
# plt.show()
plt.savefig('predlabelmodelmetriccomparison.jpg')

In [ ]:

ax = df[["XLNet (T5)",
           "XLNet (Pegasus)",
           "TF-IDF (T5)",
           "TF-IDF (Pegasus)"]].plot.bar(rot=0)
ax.set_ylim(ymin=0.1)
ax.figure.set_size_inches(6, 4)
plt.title('Metric Comparison')
# plt.show()
plt.savefig('predlabelevalmodelmetriccomparison.jpg')

In [ ]:
ax = df[["XLNet Pred. Labels (T5)", "XLNet Pred. Labels (Pegasus)", "XLNet Pred. Labels (Full Article)"]].plot.bar(rot=0)
ax.set_ylim(ymin=0.3)
ax.figure.set_size_inches(7, 6)
plt.title('Metric Comparison')

In [ ]:
ax = df[["TF-IDF Pred. Labels (T5)", "TF-IDF Pred. Labels (Pegasus)", "TF-IDF Pred. Labels (Full Article)"]].plot.bar(rot=0)
ax.set_ylim(ymin=0.3)
ax.figure.set_size_inches(7, 6)
plt.title('Metric Comparison')

In [ ]:
def calculateproportions(df, colname):
    biases = df[df[colname] == 1]
    neutrals = df[df[colname] == 0]
    length = df.shape[0]
    return biases.shape[0]/length, neutrals.shape[0]/length

In [ ]:

realbiasproportions, realneutralproportions = calculateproportions(data, "test_labels")
xlnett5biasproportions, xlnett5neutralproportions = calculateproportions(data, "t5_xlnet_predicted_labels")
xlnetpbiasproportions, xlnetpneutralproportions = calculateproportions(data, "pegasus_xlnet_predicted_labels")
xlnetbiasproportions, xlnetneutralproportions = calculateproportions(data, "XLNet")
tft5biasproportions, tft5neutralproportions = calculateproportions(data, "t5_tfidf_predicted_labels")
tfpbiasproportions, tfpneutralproportions = calculateproportions(data, "pegasus_tfidf_predicted_labels")
tfbiasproportions, tfneutralproportions = calculateproportions(data, "TFIDF")

# namelist = ["XLNet Pred. Labels (T5)",
#            "XLNet Pred. Labels (Pegasus)",
#            "XLNet Pred. Labels (Full Article)",
#            "TF-IDF Pred. Labels (T5)",
#            "TF-IDF Pred. Labels (Pegasus)",
#            "TF-IDF Pred. Labels (Full Article)"]

# t5_xlnet_predicted_labels'] = t5_xlnet_predicted_labels
# test_section['pegasus_xlnet_predicted_labels'] = pegasus_xlnet_predicted_labels
# test_section['t5_tfidf_predicted_labels'] = t5_tfidf_predicted_labels
# test_section['pegasus_tfidf_predicted_labels'] = pegasus_tfidf_predicted_labels

biasproplist = [realbiasproportions,
               xlnett5biasproportions,
               xlnetpbiasproportions,
                xlnetbiasproportions,
                tft5biasproportions,
               tfpbiasproportions,
                tfbiasproportions]
                
neutralproplist = [realneutralproportions,
               xlnett5neutralproportions,
               xlnetpneutralproportions,
                xlnetneutralproportions,
                tft5neutralproportions,
               tfpneutralproportions,
                tfneutralproportions]
                
totalnames = ["Data"]
totalnames = totalnames + namelist

plt.figure(figsize=(6, 4))
plt.subplot(1, 2, 1)

plt.bar(range(0, len(biasproplist)), biasproplist, color = "orange", width = 0.5, tick_label = totalnames)
plt.ylim(ymin=0)
plt.title('Proportions of Biased Labels')
plt.xticks(rotation=90)

plt.subplot(1, 2, 2)
plt.bar(range(0, len(neutralproplist)), neutralproplist, width = 0.5, tick_label = totalnames)
plt.ylim(ymin=0)
plt.title('Proportions of Neutral Labels')
plt.xticks(rotation=90)
plt.savefig('modelsummarylabelproportioncomparison.jpg')


In [ ]:
data[(data["t5_xlnet_predicted_labels"] != data["pegasus_xlnet_predicted_labels"]) & (data["pegasus_xlnet_predicted_labels"]==1)]

In [ ]:
data["t5_summaries"].iloc[109]

In [ ]:
data["pegasus_summaries"].iloc[109]

In [ ]:
data[(data["t5_xlnet_predicted_labels"] != data["pegasus_xlnet_predicted_labels"])].shape

In [ ]:
data[(data["t5_xlnet_predicted_labels"] != data["pegasus_xlnet_predicted_labels"]) & (data["pegasus_xlnet_predicted_labels"]==0)]

In [ ]:
data["t5_summaries"].iloc[107]

In [ ]:
data["pegasus_summaries"].iloc[107]

In [ ]:
data.iloc[1]

In [ ]:
data["pegasus_summaries"].iloc[109]

In [ ]:
plt.hist(data["pegasus_summaries"].str.len(), bins=20)


In [ ]:
plt.hist(data["t5_summaries"].str.len(), bins=20)


In [ ]:
data[""]

In [ ]:
pegasus_biased = data[data["pegasus_xlnet_predicted_labels"]==1]

In [ ]:
def distance(x, y):
    return abs(x-y)

In [ ]:
pegasus_biased = data[data["pegasus_xlnet_predicted_labels"]==1]
pegasus_bias_distance = pegasus_biased.apply(lambda row: distance(row["democrat.bias"], row["republican.bias"]), axis = 1) 
plt.hist(pegasus_bias_distance)

In [ ]:
pegasus_biased["democrat.bias"].hist()

In [ ]:
pegasus_biasedplot = pegasus_biased["democrat.bias"].value_counts()/len(real_biased)
pegasus_biasedplot

In [ ]:
real_biased = data[data["test_labels"]==1]
real_bias_distance = real_biased.apply(lambda row: distance(row["democrat.bias"], row["republican.bias"]), axis = 1) 
plt.hist(real_bias_distance)
real_biasedplot = real_biased["democrat.bias"].value_counts()/len(real_biased)
real_biasedplot

In [ ]:
pegasus_biased.biascategories.hist()

In [ ]:
pegasus_biased.text.iloc[2]

In [ ]:
pegasus_biased.iloc[2]

In [ ]:
data.head()

In [ ]:
plt.figure(figsize=(8, 8))

plt.subplot(1, 2, 1)
plt.bar()


(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
# plt.show()

In [ ]:
bertmodeldata = pd.read_table("bertmodeldata.csv", sep='\t', index_col=0)
robertamodeldata = pd.read_table("robertamodeldata.csv", sep='\t', index_col=0)
# frozenrobertamodeldata = pd.read_table("frozenrobertamodeldata.csv", sep='\t', index_col=0)
xlnetmodeldata = pd.read_table("xlnetmodeldata.csv", sep='\t', index_col=0)
# frozenxlnetmodeldata = pd.read_table("frozenxlnetmodeldata.csv", sep='\t', index_col=0)
albertmodeldata = pd.read_table("albertmodeldata.csv", sep='\t', index_col=0)


In [ ]:
bert_val_acc = bertmodeldata['ValAccuracy']
roberta_val_acc = robertamodeldata['ValAccuracy']
xlnet_val_acc = xlnetmodeldata['ValAccuracy']
albert_val_acc = albertmodeldata['ValAccuracy']

bert_val_loss = bertmodeldata['Val_loss']
roberta_val_loss = robertamodeldata['Val_loss']
# frozenroberta_val_loss = frozenrobertamodeldata['Val_loss']
xlnet_val_loss = xlnetmodeldata['Val_loss']
# frozenxlnet_val_loss = frozenxlnetmodeldata['Val_loss']
albert_val_loss = albertmodeldata['Val_loss']

epochs_range = range(25)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, bert_val_acc, label='BERT')
plt.plot(epochs_range, roberta_val_acc, label='RoBERTa')
# plt.plot(epochs_range, frozenroberta_val_acc, label='RoBERTa (6 layers trained)')
plt.plot(epochs_range, xlnet_val_acc, label='XLNet')
plt.plot(epochs_range, albert_val_loss, label='ALBERT')
plt.legend(loc='lower right')
plt.title('Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, bert_val_loss, label='BERT')
plt.plot(epochs_range, roberta_val_loss, label='RoBERTa')
# plt.plot(epochs_range, frozenroberta_val_loss, label='RoBERTa (6 layers trained)')
plt.plot(epochs_range, xlnet_val_loss, label='XLNet')
plt.plot(epochs_range, albert_val_loss, label='ALBERT')
plt.legend(loc='upper right')
plt.title('Validation Loss')
# plt.show()
plt.savefig('modeltrainingcomparison.jpg')

In [ ]:
# plot accuracy

In [ ]:
def findcommonlabel(predictionlist):
    labellist = []
    for i in range(len(predictionlist[1])):
        allin = True
        labeli = predictionlist[0][i]
        for ii in range(len(predictionlist)):
            if predictionlist[ii][i] !=labeli:
                allin = False
        if allin == True:
            labellist.append(i)
    return labellist

In [ ]:
commonpred = findcommonlabel(predictions)
len(commonpred)

In [ ]:
test_section.reset_index(inplace=True)
test_section.head()

In [ ]:
commonpreddata = test_section.iloc[commonpred]
commonpreddata.shape

In [ ]:
commonpreddata["label"].value_counts().plot.bar()

In [ ]:
plt.hist(commonpreddata.length)


In [ ]:
mixedpreddata = test_section.drop(commonpred, errors='ignore')
mixedpreddata.shape

In [ ]:
mixedpreddata["label"].value_counts().plot.bar()

In [ ]:
plt.hist(mixedpreddata.length)


In [ ]:
for i in range(len(names)):
    test_section[names[i]] = predictions[i]
test_section['encodedlabel'] = test_labels
test_section.head()

In [ ]:
def calculateproportions(df, colname):
    biases = df[df[colname] == 0]
    neutrals = df[df[colname] == 1]
    length = df.shape[0]
    return biases.shape[0]/length, neutrals.shape[0]/length

In [ ]:
realbiasproportions, realneutralproportions = calculateproportions(test_section, "encodedlabel")
bertbiasproportions, bertneutralproportions = calculateproportions(test_section, "BERT")
robertabiasproportions, robertaneutralproportions = calculateproportions(test_section, "RoBERTa")
xlnetbiasproportions, xlnetneutralproportions = calculateproportions(test_section, "XLNet")
albertbiasproportions, albertneutralproportions = calculateproportions(test_section, "ALBERT")

biasproplist = [realbiasproportions,
               bertbiasproportions,
               robertabiasproportions,
                xlnetbiasproportions,
                albertbiasproportions]
neutralproplist = [realneutralproportions,
               bertneutralproportions,
               robertaneutralproportions,
                xlnetneutralproportions,
                albertneutralproportions]
                
totalnames = ["Data", "BERT","RoBERTa", "XLNet", "ALBERT"]
plt.bar(range(0, len(biasproplist)), biasproplist, width = 0.25, tick_label = totalnames)
plt.ylim(ymin=0.3)
plt.title('Proportions of Biased Labels')



In [ ]:
plt.bar(range(0, len(neutralproplist)), neutralproplist, width = 0.25, tick_label = totalnames)
plt.ylim(ymin=0.5)
plt.title('Proportions of Neutral Labels')



In [ ]:
test_section[:10]

In [ ]:
realneutraldata = test_section[(test_section["encodedlabel"] ==1)
            & (test_section["BERT"]== 0)
             & (test_section["RoBERTa"]== 0)
             & (test_section["XLNet"]== 0)
             & (test_section["ALBERT"]== 0)]
realneutraldata

In [ ]:
test_section.iloc[9].url

In [ ]:
test_section.iloc[9].article

In [ ]:
xlnetbiasvsneutral = test_section[(test_section["encodedlabel"] ==1)
             & (test_section["XLNet"]== 0)]
xlnetbiasvsneutral

In [ ]:
realbiasdata = test_section[(test_section["encodedlabel"] ==0)
            & (test_section["BERT"]== 1)
             & (test_section["RoBERTa"]== 1)
             & (test_section["XLNet"]== 1)]
realbiasdata

In [ ]:
test_section[(test_section["encodedlabel"] ==0)
            & (test_section["XLNet"]== 1)]

In [ ]:
breitbartdata = pd.read_table("biaseddata.csv", sep='\t', index_col=0)


In [ ]:
breitbartdata[breitbartdata['article']=='on sunday evening, cbs news’ 60 minutes featured an investigative report on the benghazi attacks by lara logan. logan’s reporting confirmed that the obama administration had been warned, months in advance, about inadequate security at the u.s. facility in benghazi, and that it knew the story about a youtube video was untrue. it was a reversal for cbs news, which played a key role in the benghazi cover-up in 2012.  a year ago, cbs news released a previously unaired clip of an interview for 60 minutes with president barack obama on sep. 12, the day after the benghazi attack, in which the president suggests clearly that the attack on the u.s. consulate was premeditated. the interview contradicted obama’s subsequent claims that the attack had been a response to an anti-islam youtube video, repeated to the public for several days.  cbs news had withheld that portion of the sep. 12 interview until oct. 19, choosing instead to release a portion in which obama criticized rival mitt romney’s condemnation of the administration’s response to events–a repeated theme as the media helped obama deflect responsibility. romney had attacked the white house’s apology for the video during the attack on the cairo embassy, not knowing of the deaths in libya.  after the election, kroft continued to insist that there was “no information that anything’s been covered up” by the media or the obama administration about the benghazi attacks. that statement was wrong based on what was known at the time, and looks even worse today, after whistleblowers testified to congress earlier this year they were pressured by clinton’s chief of staff not to cooperate with congressional investigators.  logan’s investigation featured an interview with one of those whistleblowers, greg hicks, who had been the deputy to slain u.s. ambassador chris stevens. logan also interviewed “morgan jones” (pseudonym), a former british soldier who had been advising the u.s. on security in benghazi and had warned the state department that libyan guards being trained to guard the compound were not up to the job.  morgan’s warnings came to fruition on sep. 11, 2012, when he saw the guards disperse after being told by the attackers: “we’re here to kill americans, not libyans.” morgan did what little he could to stop the attack. a second attack at the cia annex was partly fought off by a small group that saved five american lives and has since, logan reported, earned “the distinguished service cross and the navy cross” for its bravery.  yet the obama administration did not send any assistance throughout the night, and is thought to have issued a stand-down order to prevent any intervention. what logan’s report makes clear is that there is no way that president obama or his secretary of state could plausibly claim that a youtube video had inspired a spontaneous act of violence on that scale against the u.s. consulate, the cia annex and their personnel.  that story was a lie when it was told–and it was a lie aided and abetted by the mainstream media, including cbs news and 60 minutes, which could have exploded the administration’s youtube video alibi even as obama and clinton were telling it to the families of the victims, to the voting public, and to the world. the sunday night report is commendable, but as an attempt to atone for malpractice, it is far too little, too late.']

In [ ]:
data.head()

In [ ]:
# Metrics in comparison to XLNet predicted article labels

In [ ]:
t5_xlnet_predicted_labels = data['t5_xlnet_predicted_labels']
pegasus_xlnet_predicted_labels = data['pegasus_xlnet_predicted_labels']
t5_tfidf_predicted_labels = data['t5_tfidf_predicted_labels']
pegasus_tfidf_predicted_labels = data['pegasus_tfidf_predicted_labels']
test_labels = data['XLNet'] 

In [ ]:
predlist = [t5_xlnet_predicted_labels,
           pegasus_xlnet_predicted_labels,
           t5_tfidf_predicted_labels,
           pegasus_tfidf_predicted_labels
           ]
namelist = ["XLNet (T5)",
           "XLNet (Pegasus)",
           "TF-IDF (T5)",
           "TF-IDF (Pegasus)"]

In [ ]:
modelaccuracylist = []
modelnamelist = []
modelprecisionlist = []
modelrecalllist = []
modelf1list = []
mlist = []
print("Metrics for Biased as baseline class (0):")
print()
for i in range(len(predlist)):
    test_classes = predlist[i]
    modelname = namelist[i]
    modelnamelist.append(modelname)
    print(modelname +" Metrics:")
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(test_labels, test_classes)
    print('Accuracy: %f' % accuracy)
    modelaccuracylist.append(accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(test_labels, test_classes)
    print('Precision: %f' % precision)
    modelprecisionlist.append(precision)
    # recall: tp / (tp + fn)
    recall = recall_score(test_labels, test_classes)
    print('Recall: %f' % recall)
    modelrecalllist.append(recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(test_labels, test_classes)
    modelf1list.append(f1)
    print('F1 score: %f' % f1)
    mlist.append([accuracy, precision, recall, f1])
#     print(classification_report(test_labels,test_classes))
    print("")
 

In [ ]:
df = pd.DataFrame(columns = modelnamelist)
df.loc['Accuracy'] = modelaccuracylist
df.loc['Precision'] = modelprecisionlist
df.loc['Recall'] = modelrecalllist
df.loc['F1'] = modelf1list
df.head()

In [ ]:
ax = df.plot.bar(rot=0)
ax.set_ylim()
ax.figure.set_size_inches(6.5, 7)
plt.title('Predicted Summary Labels Compared to XLNet-Predicted Article Labels')
# plt.show()
plt.savefig('predlabelmodelxlnetmetriccomparison.jpg')

In [ ]:
# Metrics in comparison to TF-IDF predicted article labels

In [ ]:
t5_xlnet_predicted_labels = data['t5_xlnet_predicted_labels']
pegasus_xlnet_predicted_labels = data['pegasus_xlnet_predicted_labels']
t5_tfidf_predicted_labels = data['t5_tfidf_predicted_labels']
pegasus_tfidf_predicted_labels = data['pegasus_tfidf_predicted_labels']
test_labels = data['TFIDF'] 

In [ ]:
predlist = [t5_xlnet_predicted_labels,
           pegasus_xlnet_predicted_labels,
           t5_tfidf_predicted_labels,
           pegasus_tfidf_predicted_labels
           ]
namelist = ["XLNet (T5)",
           "XLNet (Pegasus)",
           "TF-IDF (T5)",
           "TF-IDF (Pegasus)"]

In [ ]:
modelaccuracylist = []
modelnamelist = []
modelprecisionlist = []
modelrecalllist = []
modelf1list = []
mlist = []
print("Metrics for Biased as baseline class (0):")
print()
for i in range(len(predlist)):
    test_classes = predlist[i]
    modelname = namelist[i]
    modelnamelist.append(modelname)
    print(modelname +" Metrics:")
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(test_labels, test_classes)
    print('Accuracy: %f' % accuracy)
    modelaccuracylist.append(accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(test_labels, test_classes)
    print('Precision: %f' % precision)
    modelprecisionlist.append(precision)
    # recall: tp / (tp + fn)
    recall = recall_score(test_labels, test_classes)
    print('Recall: %f' % recall)
    modelrecalllist.append(recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(test_labels, test_classes)
    modelf1list.append(f1)
    print('F1 score: %f' % f1)
    mlist.append([accuracy, precision, recall, f1])
#     print(classification_report(test_labels,test_classes))
    print("")
 

In [ ]:
df = pd.DataFrame(columns = modelnamelist)
df.loc['Accuracy'] = modelaccuracylist
df.loc['Precision'] = modelprecisionlist
df.loc['Recall'] = modelrecalllist
df.loc['F1'] = modelf1list
df.head()

In [ ]:
ax = df.plot.bar(rot=0)
ax.set_ylim()
ax.figure.set_size_inches(6.5, 7)
plt.title('Predicted Summary Labels Compared to TF-IDF-Predicted Article Labels')
# plt.show()
plt.savefig('predlabelmodelTFIDFmetriccomparison.jpg')